# Build an End-to-End System

This puts together the chain of prompts that you saw throughout the course.

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [7]:
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension(raw_css=[
    ".assistant{background-color:#F6F6F6;padding:8px;border-radius:4px;color:#000000;}",
    ".chat-scroll{max-height:300px;overflow:auto;padding-right:6px;}"  # <-- agrega esto
])


from pathlib import Path
# Agregar la raíz del proyecto al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from lib_openai import get_completion_from_messages


## System of chained prompts for processing the user query

In [2]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    #response = openai.Moderation.create(input=user_input)
    #moderation_output = response["results"][0]

    #if moderation_output["flagged"]:
    #    print("Step 1: Input flagged by Moderation API.")
    #    return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")
    print(product_information)
    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]
    print(final_response)

    # Step 5: Put the answer through the Moderation API
    #response = openai.Moderation.create(input=final_response)
    #moderation_output = response["results"][0]

    #if moderation_output["flagged"]:
    #    if debug: print("Step 5: Response flagged by Moderation API.")
    #    return "Sorry, we cannot provide this information."

    #if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")
    print(evaluation_response)

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages


In [3]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
[{'products': ['SmartX ProPhone']}, {'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]
[{'products': ['SmartX ProPhone']}, {'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]
Step 2: Extracted list of products.
Step 3: Looked up product information.
{
    "name": "SmartX ProPhone",
    "category": "Smartphones and Accessories",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "A powerful smartphone with advanced camera features.",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "Cameras and Camcorders",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
    

In [4]:
user_input = "show me 5 smartphones with the best cameras"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
[]
[]
Step 2: Extracted list of products.
Step 3: Looked up product information.

Step 4: Generated response to user question.
Here are 5 smartphones known for having the best cameras:

1. iPhone 15 Pro Max – Excellent all-around camera with great low-light performance.
2. Samsung Galaxy S23 Ultra – High-resolution sensor and versatile zoom capabilities.
3. Google Pixel 8 Pro – Outstanding computational photography and color accuracy.
4. Sony Xperia 1 V – Great for photography enthusiasts with manual controls.
5. OnePlus 11 Pro – Strong camera system with good image processing.

Are you looking for any specific features like video recording quality or budget range?
Step 6: Model evaluated the response.
Yes, the response sufficiently answers the question by listing 5 smartphones with the best cameras and providing brief highlights of their camera strengths. Additionally, it asks a relevant follow-up question about specific features or budget, which

### Function that collects user and assistant messages over time

In [5]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=True)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=500)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=500, css_classes=['assistant'])))
 
    return pn.Column(*panels)

### Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [8]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Send", button_type="primary")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.panel(interactive_conversation, loading_indicator=True, css_classes=['chat-scroll']),  # conversación arriba
    pn.Row(inp, button_conversation, sizing_mode='stretch_width')  # input al final
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'5843f66b-9afd-4c5e-bf54-aab64a9373c2': {'version…